In [1]:
import matplotlib.pyplot as plt
import cv2
from functions import *

In [14]:
labels = [
    ['1_real'],
    ['50_cents'],
    ['25_cents'],
    ['5_cents'],
    ['10_cents']
]

training_data = list()

for i in range(5):
    img_gray, img_rgb = prepare_img(f'images/img{i}.png')
    circles = find_circles(img_gray)
    circles = treat_circles(circles)
    
    if circles is not None:
        filtered_circles = filter_inner_circles(circles, 10)
        
        # Get only the coins from the image
        coins = [extract_coin(img_rgb, circle) for circle in filtered_circles]
        
        # Apply SIFT to the coins
        sift_results = [extract_sift_features(coin, 128) for coin in coins]
        descriptors = [result[1] for result in sift_results]
        
        # Create visual vocabulary
        visual_vocabulary = create_visual_vocabulary(descriptors, 100)
        
        # Get coins histogram of visual words
        coins_histograms = [calculate_histogram_of_visual_words(descriptor, visual_vocabulary) for descriptor in descriptors]
        
        # Add coin histograms and their corresponding labels to the training data
        for histogram, label in zip(coins_histograms, labels[i]):
            training_data.append((histogram, label))
            
for i in training_data:
    print(i)

(array([1, 1, 3, 4, 1, 1, 2, 2, 2, 1, 1, 1, 3, 1, 2, 1, 2, 1, 1, 1, 3, 1,
       1, 3, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), '1_real')
(array([3, 1, 1, 2, 9, 1, 4, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1,
       3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]), '50_cents')
(array([1, 2, 7, 1, 2, 2, 1, 1, 2, 1, 1, 5, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1,
       1,